# OpenAI Responses API: Advanced Tweet Analysis with File & Web Search Integration

## What is the OpenAI Responses API?

The Responses API is a new API released in March 2025. It is a combination of the traditional 
Chat Completions API and the Assistants API, providing support for:

- **Traditional Chat Completions:** Facilitates seamless conversational AI experiences.
- **Web Search:** Enables real-time information retrieval from the internet.
- **File Search:** Allows searching within files for relevant data.

Accordingly, the Assistants API will be retired in 2026. 

> **For new users, OpenAI recommends using the Responses API instead of the Chat Completions API to leverage its expanded capabilities.**

For a comprehensive comparison between the Responses API and the Chat Completions API, refer to the official OpenAI documentation: 
[Responses vs. Chat Completions](https://platform.openai.com/docs/guides/responses-vs-chat-completions).

## Summary of This Notebook
This notebook provides a hands-on guide for using the **OpenAI Responses API** to analyze tweets. 
It covers essential techniques such as:

- **Connecting to a MongoDB database** to store and retrieve tweets.
- **Extracting tweets** and converting them into a structured format for further analysis.
- **Creating a vector store** and uploading tweets for semantic search.
- **Using file search** to analyze private datasets.
- **Performing web search** to retrieve the latest public information.
- **Utilizing stateful responses** to maintain conversation context.
- **Combining file and web search** to enhance retrieval-augmented generation (RAG) applications.

By the end of this notebook, users will be able to integrate OpenAI's Responses API for efficient data retrieval 
and analysis of structured and unstructured data.

## Install Required Libraries
To use the OpenAI Responses API and interact with a MongoDB database, we need to install the following libraries:

- **`openai`**: Provides access to OpenAI's APIs, including the Responses API
- **`pymongo`**: A Python driver for MongoDB to store and retrieve tweets.

In [1]:
pip install openai pymongo -q

Note: you may need to restart the kernel to use updated packages.


## Import Required Libraries

In [2]:
from IPython.display import Markdown, display
import boto3
from botocore.exceptions import ClientError
import json
import io

## Retrieve Secrets from AWS Secrets Manager

In [3]:
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Connect to MongoDB

In [4]:
import pymongo
from pymongo import MongoClient
mongodb_connect = get_secret('mongodb')['connection_string']

mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection
# tweet_collection.create_index([("tweet.id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

## Extract Tweets from MongoDB

In [5]:
filter={

    
}
project={
    'tweet.text': 1,
    '_id':0
}
#rename the client to mongo_client
result = mongo_client['demo']['tweet_collection'].find(
  filter=filter,
  projection=project
)

After retrieving tweets from MongoDB, we convert the query result into a list format for easier processing.
The data is then serialized into a JSON-formatted string, ensuring it can be properly stored and shared across different services.
Using `io.BytesIO`, we create an in-memory JSON file, eliminating the need for disk writes.
This approach is particularly useful for applications that require temporary file storage, such as uploading datasets
to OpenAI's file search API or cloud storage for further analysis.

In [6]:
result_list = list(result)

# Convert result list to JSON string
json_data = json.dumps(result_list, default=str, indent=4)

# Create an in-memory JSON file
json_bytes = io.BytesIO(json_data.encode("utf-8"))
json_bytes.name = "tweet.json" 

In [7]:
print('Number of tweets: ',len(result_list))

Number of tweets:  298


## Initialize OpenAI Client

In [8]:
from openai import OpenAI
openai_api_key  = get_secret('openai')['api_key']

client = OpenAI(api_key=openai_api_key)

## File Search API

### Introduction to File Search
File search API enables efficient retrieval of relevant information 
from uploaded files by leveraging vector-based indexing. This feature is particularly useful 
for searching large datasets, extracting insights, and improving retrieval-augmented generation (RAG) applications.

Unlike traditional keyword-based searches, the Responses API uses embeddings 
to identify semantically relevant content, making it ideal for analyzing structured 
and unstructured text data (OpenAI, 2025).

For more details, visit the official OpenAI documentation: 
[File Search in Responses API](https://platform.openai.com/docs/guides/tools-file-search).

### Create a Vector Store

In [9]:
vector_store = client.vector_stores.create(
    name="tweet_base"
)
vector_store_id = vector_store.id
# print(vector_store_id)

### Upload Tweets File

In [10]:
file = client.files.create(
            file=json_bytes,
            purpose="assistants",)

file_id = file.id
# print(file_id)

### Attach File to Vector Store

In [11]:
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

# print(attach_status.id)

### Query the Vector Store

In [25]:
query = "terrorist finanacing"

In [26]:
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

or is it a new thing of facilitating terrorism and kidnapping financing ? NOTE : the traditional sys
 Relevant score: 0.8030493017951746
Authorities confiscated more than $200,000 of cryptocurrenc\u2026"
        }
    },
    {
        "t
 Relevant score: 0.7601494564715453
In fact, it was even placed on\u2026"
        }
    },
    {
        "tweet": {
            "text": 
 Relevant score: 0.736914869362027
{
        "tweet": {
            "text": "RT @LiquidFaerie: The Qatari government has been accused o
 Relevant score: 0.734801407876858
}
    },
    {
        "tweet": {
            "text": "@Georgebsmit @Xen_dk @MiddleearthMixr Not lik
 Relevant score: 0.7271216477540839


## OpenAI Response API

### Simple Response

In [27]:
simple_response = client.responses.create(
  model="gpt-4o",
  input=[
      {
          "role": "user",
          "content": query
      }
  ]
)

In [28]:
display(Markdown(simple_response.output_text))

Terrorist financing involves the collection and provision of funds to support terrorist activities. It can use both legitimate and illegitimate sources, and funds may move through traditional banking systems, underground networks, or informal channels like hawalas. Key strategies to combat terrorist financing include:

1. **Regulation and Compliance**: Enforcing regulations on financial institutions to monitor suspicious activities and report them.

2. **International Cooperation**: Nations working together to share intelligence and track cross-border transactions.

3. **Sanctions and Asset Freezing**: Freezing assets of individuals and organizations linked to terrorism.

4. **Transparency in Charities**: Ensuring non-profits and charities are not used as fronts for financing terrorism.

5. **Capacity Building**: Assisting countries in strengthening their financial systems to detect and prevent terrorist financing.

Overall, combating terrorist financing requires a comprehensive approach involving legal, financial, and administrative measures.

### File Search Response

In [29]:

file_search_response = client.responses.create(
    input= query,
    model="gpt-4o",
    temperature = 0,
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    }]
)

In [30]:
display(Markdown(file_search_response.output_text))


The document contains several mentions of terrorist financing, including accusations against entities like Binance for allegedly facilitating terrorism and kidnapping financing through illicit fund transfers. Additionally, there are claims about various governments and organizations being involved in funding terrorism, such as accusations against the Qatari government for financing terrorist groups. There are also discussions about the legal and regulatory challenges related to combating terrorism financing, including the role of cryptocurrency and the need for compliance with anti-money laundering (AML) and counter-terrorism financing (CTF) regulations.

## Web Search API

### Introduction to Web Search
The OpenAI Web Search tool allows models to retrieve real-time information from the internet. 
This capability is particularly useful for obtaining up-to-date data, fact-checking, and expanding knowledge 
without relying solely on pre-trained information. 

By leveraging OpenAI's web search functionality, the Responses API can fetch external data 
and provide accurate, relevant results in real time (OpenAI, 2025). 
This feature enhances applications that require the latest insights, such as news aggregation, research, 
or dynamic content generation.

For more details, visit the official OpenAI documentation: 
[Web Search in Responses API](https://platform.openai.com/docs/guides/tools-web-search).

### Perform Web Search

In [31]:
web_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [32]:
display(Markdown(web_search_response.output_text))

Terrorist financing involves providing funds or financial support to individuals or groups engaged in terrorist activities. This can occur through various means and channels, including:

1. **Criminal Activities**: Profits from criminal endeavors like drug trafficking, smuggling, and robbery may be used to fund terrorism.

2. **Donations and Charities**: Funds may be raised through charitable organizations, sometimes without the knowledge of donors, to support terrorist activities.

3. **Business Operations**: Legitimate businesses may be used to launder money and funnel resources to terrorist groups.

4. **State Sponsorship**: Some governments may directly or indirectly support terrorist organizations financially.

5. **Online Platforms**: Crowdfunding, cryptocurrency, and online fundraising can be exploited to gather funds without revealing the source.

To combat this, worldwide efforts include stringent regulations, monitoring financial transactions, and collaborating internationally to trace and disrupt funding channels.

If you have specific questions or a particular aspect you're interested in, feel free to let me know!

### Stateful Response

The OpenAI Responses API includes a stateful feature that enables continuity in interactions. 
By using the `response_id`, a conversation can persist across multiple queries, 
allowing users to refine or expand upon previous searches. This is particularly useful for iterative research, 
dynamic content generation, and applications that require follow-up queries based on prior responses.

In [33]:
fetched_response = client.responses.retrieve(response_id=web_search_response.id)
display(Markdown(fetched_response.output_text[:100]))

Terrorist financing involves providing funds or financial support to individuals or groups engaged i

### Continue Query with Web Search

In [34]:
continue_query = 'find different news'

continue_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= continue_query,
    previous_response_id=web_search_response.id,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [35]:
display(Markdown(continue_search_response.output_text))

Recent developments in combating terrorist financing include:

**Lebanon's Central Bank Initiatives**

Karim Souaid, the newly appointed governor of Lebanon's central bank, has prioritized combating money laundering and terrorist financing. This focus addresses concerns from international bodies like the Financial Action Task Force (FATF) and the United States. Lebanon was placed on the FATF’s grey list in 2024, impacting investor confidence amid efforts to recover from a severe financial crisis. Souaid's agenda includes identifying influential individuals, maintaining the bank's independence, rescheduling public debt, repaying depositors, and encouraging private banks to increase capital or consider mergers. Non-compliant banks may face liquidation, with protections for depositors. ([reuters.com](https://www.reuters.com/world/middle-east/lebanon-central-bank-must-fight-money-laundering-terrorist-financing-new-2025-04-04/?utm_source=openai))

**U.S. Sanctions on Sinaloa Cartel Associates**

The U.S. Treasury has sanctioned six individuals and seven companies for allegedly laundering money for the Sinaloa Cartel, now designated a foreign terrorist organization. These entities reportedly used front companies and shell corporations, including currency exchange businesses at the U.S.-Mexico border, to facilitate money laundering. The sanctions aim to disrupt the cartel's financial operations, particularly its trafficking of fentanyl into the United States. ([apnews.com](https://apnews.com/article/e976f2219e4588281d19d925ced9e306?utm_source=openai))

**Conviction in Virginia for ISIS Financing**

Mohammed Chhipa, a 35-year-old from Springfield, Virginia, was convicted on terrorism charges for providing material support to the Islamic State group. In an FBI sting operation, Chhipa accepted and funneled funds intended for the group, converting cash into bitcoin and transferring it to accounts in Turkey. He aimed to help women associated with ISIS escape prison camps. Chhipa, married to Allison Fluke-Ekren (serving a 20-year sentence for her involvement with ISIS), will be sentenced in May. ([apnews.com](https://apnews.com/article/1bb283f4d944bddba1d4be55d922b179?utm_source=openai))

**FinCEN's Measures Against Al-Huda Bank**

The U.S. Department of the Treasury’s Financial Crimes Enforcement Network (FinCEN) has issued a final rule severing Al-Huda Bank from the U.S. financial system. The Iraq-based bank has been identified as a conduit for terrorist financing, supporting organizations like Iran’s Islamic Revolutionary Guard Corps and affiliated militias. The rule prohibits U.S. financial institutions from maintaining correspondent accounts for Al-Huda Bank, aiming to protect the U.S. financial system from illicit activities. ([fincen.gov](https://www.fincen.gov/news/news-releases/fincen-finalizes-financial-measure-against-iraq-based-al-huda-bank-combat?utm_source=openai))

**Treasury's Focus on Cryptocurrency and Terrorist Financing**

The U.S. Treasury Department has emphasized the need for digital asset companies to prevent terrorists from accessing resources through cryptocurrencies. Deputy Secretary Wally Adeyemo stated that if these companies do not act to prevent illicit financial flows, the U.S. and its partners will intervene. This follows concerns about crypto intermediaries facilitating illicit finance activities, including support for Hamas' attacks on Israel. ([blockworks.co](https://blockworks.co/news/treasury-crypto-terrorist-financing?utm_source=openai))


## Recent Actions Against Terrorist Financing:
- [Lebanon central bank must fight money laundering, terrorist financing, new governor says](https://www.reuters.com/world/middle-east/lebanon-central-bank-must-fight-money-laundering-terrorist-financing-new-2025-04-04/?utm_source=openai)
- [US sanctions Sinaloa Cartel associates for alleged money laundering](https://apnews.com/article/e976f2219e4588281d19d925ced9e306?utm_source=openai)
- [Virginia man convicted of funneling money to Islamic State group](https://apnews.com/article/1bb283f4d944bddba1d4be55d922b179?utm_source=openai) 

### Combining File Search and Web Search

This is an example of using file search to analyze private data and web search to retrieve public or the latest data. 
The Responses API allows developers to integrate these tools to enhance retrieval-augmented generation (RAG) applications. 
By combining file search with web search, users can leverage structured internal knowledge while also retrieving real-time 
information from external sources, ensuring comprehensive and up-to-date responses. 

In [36]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Retrieve the results from the file search first, and use the web search tool to expand the results with news resources",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)

In [37]:
display(Markdown(combined_search_response.output_text))

The tweets in the uploaded file discuss various aspects of terrorist financing, including accusations against entities like Binance and the Qatari government. Here are some key points:

1. **Accusations Against Binance**: The Nigerian government has accused Binance of facilitating terrorism and kidnapping financing through illicit fund transfers on its cryptocurrency exchange platform. Binance is facing legal challenges, including an $81 billion lawsuit.

2. **Qatari Government Accusations**: The Qatari government has been accused of funding and financing terrorism and terrorist groups, including Libyan Islamist militias, Syrian jihadist groups, Hamas, the al-Nusrah Front, and the Islamic State.

3. **General Concerns**: There are broader concerns about the use of cryptocurrencies in financing terrorism, with discussions around the need for regulations and compliance with anti-money laundering (AML) and counter-terrorism financing (CTF) regulations.

These points highlight ongoing global concerns and legal actions related to the financing of terrorism, particularly through digital and decentralized financial systems.